In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Training Dataset
data = '/content/drive/MyDrive/UNSW_2018_IoT_Botnet_Final_10_Best.csv'
df = pd.read_csv(data, sep=';')

df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,pkSeqID,proto,saddr,sport,daddr,dport,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,attack,category,subcategory
0,0,1,tcp,192.168.100.147,49960,192.168.100.7,80,9,0.068909,75,0.000000,1,0.068909,96,14.511893,0.566862,0.137818,1,DoS,HTTP
1,1,2,arp,192.168.100.7,-1,192.168.100.147,-1,10,0.000000,2,0.000131,2,0.000131,1,0.000000,0.000000,0.000131,1,DoS,HTTP
2,2,3,tcp,192.168.100.147,49962,192.168.100.7,80,11,0.064494,75,0.000000,1,0.064494,96,15.505319,0.567549,0.128988,1,DoS,HTTP
3,3,4,tcp,192.168.100.147,49964,192.168.100.7,80,12,0.064189,75,0.000000,1,0.064189,96,15.578993,0.567570,0.128378,1,DoS,HTTP
4,4,5,tcp,192.168.100.147,49966,192.168.100.7,80,13,0.063887,75,0.000000,1,0.063887,96,15.652637,0.567630,0.127774,1,DoS,HTTP


In [7]:
# data 정보 출력
# 각 컬럼별 데이터 수, 데이터 타입 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3668522 entries, 0 to 3668521
Data columns (total 20 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   pkSeqID            int64  
 2   proto              object 
 3   saddr              object 
 4   sport              object 
 5   daddr              object 
 6   dport              object 
 7   seq                int64  
 8   stddev             float64
 9   N_IN_Conn_P_SrcIP  int64  
 10  min                float64
 11  state_number       int64  
 12  mean               float64
 13  N_IN_Conn_P_DstIP  int64  
 14  drate              float64
 15  srate              float64
 16  max                float64
 17  attack             int64  
 18  category           object 
 19  subcategory        object 
dtypes: float64(6), int64(7), object(7)
memory usage: 559.8+ MB


In [8]:
# 3,668,522개의 데이터, 20개의 특성
df.shape

(3668522, 20)

In [9]:
# 수치형 변수 분포 확인
df.describe()

,Unnamed: 0,pkSeqID,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,attack
count,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06
mean,1.834260e+06,1.834262e+06,1.213204e+05,8.871499e-01,8.253848e+01,1.017540e+00,3.134390e+00,2.231063e+00,9.245168e+01,4.455046e-01,2.955111e+00,3.020015e+00,9.998700e-01
std,1.059011e+06,1.059011e+06,7.579428e+04,8.037139e-01,2.439739e+01,1.483688e+00,1.186971e+00,1.517728e+00,1.817643e+01,6.028863e+01,7.245425e+02,1.860877e+00,1.140212e-02
min,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.171302e+05,9.171312e+05,5.487600e+04,3.001900e-02,6.900000e+01,0.000000e+00,3.000000e+00,1.819670e-01,1.000000e+02,0.000000e+00,1.559700e-01,2.806072e-01,1.000000e+00
50%,1.834260e+06,1.834262e+06,1.177690e+05,7.938960e-01,1.000000e+02,0.000000e+00,4.000000e+00,2.690125e+00,1.000000e+02,0.000000e+00,2.837830e-01,4.009111e+00,1.000000e+00
75%,2.751391e+06,2.751392e+06,1.849300e+05,1.745296e+00,1.000000e+02,2.151138e+00,4.000000e+00,3.565203e+00,1.000000e+02,0.000000e+00,4.882010e-01,4.293582e+00,1.000000e+00
max,3.668521e+06,3.668522e+06,2.622120e+05,2.496763e+00,1.000000e+02,4.980471e+00,1.100000e+01,4.981882e+00,1.000000e+02,5.882353e+04,1.000000e+06,4.999999e+00,1.000000e+00


In [10]:
# 타겟 종류 확인
df['category'].unique()

array(['DoS', 'DDoS', 'Normal', 'Reconnaissance', 'Theft'], dtype=object)

In [11]:
# 속성과 클래스 분리

import numpy as np

data = df[['seq', 'stddev', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'drate', 'srate', 'max']].to_numpy() 

# category: 공격의 종류
target = df['category'].to_numpy()  # category: 공격의 종류

In [12]:
print(data.shape)  # 총 3,668,522개의 데이터, 10개의 특성

(3668522, 10)


In [13]:
data

array([[9.0000000e+00, 6.8909000e-02, 9.6000000e+01, ..., 1.4511893e+01,
        5.6686200e-01, 1.3781800e-01],
       [1.0000000e+01, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 1.3100000e-04],
       [1.1000000e+01, 6.4494000e-02, 9.6000000e+01, ..., 1.5505319e+01,
        5.6754900e-01, 1.2898800e-01],
       ...,
       [8.4000000e+01, 0.0000000e+00, 1.9000000e+01, ..., 5.5665690e+00,
        4.5228370e+00, 2.8743020e+00],
       [8.5000000e+01, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 3.0000000e-06],
       [1.0000000e+01, 5.6000000e-05, 2.0000000e+00, ..., 8.2550000e-03,
        8.2550000e-03, 2.7200000e-04]])

In [14]:
target

array(['DoS', 'DoS', 'DoS', ..., 'Theft', 'Theft', 'Theft'], dtype=object)

In [15]:
# Train-test Split
## 훈련셋 : 테스트셋 = 8 : 2

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [16]:
#훈련세트
print("train_input: ", train_input.shape)
print("train_target: ", train_target.shape)

#테스트세트
print("test_input: ", test_input.shape)
print("test_target: ", test_target.shape)

train_input:  (2934817, 10)
train_target:  (2934817,)
test_input:  (733705, 10)
test_target:  (733705,)


In [17]:
#Data 전처리
## 최소최대정규화(MinMaxScaler) 사용

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_input)

train_scaled = scaler.transform(train_input)
test_scaled = scaler.transform(test_input)


In [18]:
#ML 모델 훈련 : 랜덤포레스트

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42) 
scores = cross_validate(rf, train_input, train_target,
        return_train_score=True, n_jobs=-1) 

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9999997444474463 0.9999625189565797


In [19]:
#랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것.
#앞의 랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도를 출력

rf.fit(train_input, train_target) 
print(rf.feature_importances_)

[0.04631508 0.07868486 0.19379143 0.08946451 0.06067042 0.11427968
 0.07970807 0.00940777 0.24545112 0.08222707]


In [20]:
#OOB샘플
#부트스트랩 샘플로 훈련한 결정트리를평가(검증세트의 역할)

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) 
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.9999713781131839


**DDos Botnet Attack Dataset으로** **예측해보기**

In [22]:
# 구글 드라이브에서 ddos csv 파일 가져오기
import pandas as pd
filename = '/content/drive/MyDrive/DDoSdata.csv'

ddos = pd.read_csv(filename)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
ddos.head()

,Unnamed: 0,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1650261,1650261,1.528103e+09,e,1,tcp,1,192.168.100.150,54110,192.168.100.3,...,1.21662,40,38,1.56093,1.21662,328,308,1,DDoS,HTTP
1,1650262,1650262,1.528103e+09,e,1,tcp,1,192.168.100.150,54112,192.168.100.3,...,1.21662,40,38,1.56107,1.21662,328,308,1,DDoS,HTTP
2,1650263,1650263,1.528103e+09,e,1,tcp,1,192.168.100.150,54114,192.168.100.3,...,1.21662,40,38,1.24980,1.21662,328,308,1,DDoS,HTTP
3,1650264,1650264,1.528103e+09,e,1,tcp,1,192.168.100.150,54116,192.168.100.3,...,1.21662,40,38,1.24986,1.21662,328,308,1,DDoS,HTTP
4,1650265,1650265,1.528103e+09,e,1,tcp,1,192.168.100.150,54118,192.168.100.3,...,1.21662,40,38,1.24991,1.21662,328,308,1,DDoS,HTTP


In [24]:
# 타겟 종류 확인
ddos['category'].unique()

array(['DDoS', 'Normal'], dtype=object)

In [25]:
# 속성과 클래스 분리
import numpy as np

ddos_data = ddos[['seq', 'stddev', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'drate', 'srate', 'max']].to_numpy()  # 특성 논문 참고하여 임의로 선택, 후에 이유 덧붙여서 얘기하기

# category: 공격의 종류
ddos_target = ddos['category'].to_numpy()

In [26]:
ddos_data

array([[2.000000e+01, 5.441260e-01, 4.000000e+01, ..., 4.682800e-01,
        7.804670e-01, 1.223599e+00],
       [2.100000e+01, 5.441970e-01, 4.000000e+01, ..., 4.683220e-01,
        7.805360e-01, 1.223769e+00],
       [2.200000e+01, 1.110847e+00, 4.000000e+01, ..., 9.002140e-01,
        6.248990e-01, 2.221694e+00],
       ...,
       [3.815000e+03, 0.000000e+00, 1.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 2.778960e-01],
       [3.816000e+03, 0.000000e+00, 2.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 2.807420e-01],
       [3.817000e+03, 0.000000e+00, 2.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 2.805130e-01]])

In [27]:
ddos_data.shape

(1927101, 10)

In [28]:
ddos_target

array(['DDoS', 'DDoS', 'DDoS', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

In [30]:
#훈련한 결정 트리 모델에 DDoS, Normal 트래픽만 넣어서 테스트
print(rf.score(ddos_data, ddos_target))

0.999997924343353
